In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [18,10]

In [3]:
non_numeric = ['BMI_class', 'Height_class', 'Gender', 'Component', 'Branch']
regression = ['BMI_class', 'Height_class', 'Gender', 'Component', 'Branch', 'BMI']

def load_ansur(cols_to_drop, test_size, model_type='class'):
    df_m = pd.read_csv('data/ANSUR_II_MALE.csv')
    df_f = pd.read_csv('data/ANSUR_II_FEMALE.csv')
    ansur_df = pd.concat([df_m, df_f], axis=0)
    
    if model_type == 'class': 
        X = ansur_df.drop(non_numeric, axis=1)
        y = ansur_df['Gender']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    elif model_type == 'reg':
        X = ansur_df.drop(regression, axis=1)
        y = ansur_df["BMI"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
    else:
        print('please specify model type')
        
    return X, y, X_train, X_test, y_train, y_test

In [5]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, 0.3)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

### Creating a logistic regression model

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(X_train_std, y_train)

X_test_std = scaler.transform(X_test)

In [7]:
y_pred = lr.predict(X_test_std)
print(accuracy_score(y_test, y_pred))

1.0


### Inspecting the feature coefficients 

In [8]:
print(lr.coef_[0][:10])

[ 0.11968814  0.14414063 -0.08561985 -0.28054672 -0.05632391 -0.0621446
  0.14700426  0.73677963  0.7013665  -0.83836831]


In [9]:
coef_dict = dict(zip(X.columns, abs(lr.coef_[0])))

{k: v for i, (k, v) in enumerate(coef_dict.items()) if i < 10}

{'abdominalextensiondepthsitting': 0.11968813731025733,
 'acromialheight': 0.1441406275554733,
 'acromionradialelength': 0.08561985044125861,
 'anklecircumference': 0.2805467206936949,
 'axillaheight': 0.05632390833085285,
 'balloffootcircumference': 0.06214459531742643,
 'balloffootlength': 0.14700426319715112,
 'biacromialbreadth': 0.7367796309781828,
 'bicepscircumferenceflexed': 0.7013665009994752,
 'bicristalbreadth': 0.8383683123864404}

In [10]:
low_coef = {k: v for k, v in coef_dict.items() if v < .401}

cols = [k for k, v in low_coef.items()]

In [11]:
X.drop(cols, axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lr.fit(scaler.fit_transform(X_train), y_train)

print(accuracy_score(y_test, lr.predict(scaler.transform(X_test))))

1.0


In [13]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, 0.3)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [14]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=LogisticRegression(), n_features_to_select=5, verbose=0)
rfe.fit(X_train_std, y_train)

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=5, step=1, verbose=0)

In [15]:
X.columns[rfe.support_]

Index(['biacromialbreadth', 'buttockcircumference', 'chestheight',
       'lateralfemoralepicondyleheight', 'neckcircumference'],
      dtype='object')

In [16]:
#print(dict(zip(X.columns, rfe.ranking_)))

In [17]:
print(accuracy_score(y_test, rfe.predict(X_test_std)))

0.9950576606260296


In [18]:
def load_pima(cols_to_drop):
    df = pd.read_csv('data/PimaIndians.csv')

    X = df.drop(cols_to_drop, axis=1)
    y = df['test']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return X, y, X_train, X_test, y_train, y_test

X, y, X_train, X_test, y_train, y_test = load_pima('test')

scaler = StandardScaler()
lr = LogisticRegression()

In [19]:
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)

# Fit the logistic regression model on the scaled training data
lr.fit(X_train_std, y_train)

# Scale the test features
X_test_std = scaler.transform(X_test)

# Predict diabetes presence on the scaled test set
y_pred = lr.predict(X_test_std)

# Prints accuracy metrics and feature coefficients
print("{0:.1%} accuracy on test set.".format(accuracy_score(y_test, y_pred))) 
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

75.4% accuracy on test set.
{'pregnant': 0.21, 'glucose': 1.09, 'diastolic': 0.05, 'triceps': 0.11, 'insulin': 0.2, 'bmi': 0.54, 'family': 0.19, 'age': 0.51}


In [20]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Create the RFE with a LogisticRegression estimator and 3 features to select
rfe = RFE(estimator=LogisticRegression(), n_features_to_select=3, verbose=1)

# Fits the eliminator to the data
rfe.fit(X_train_std, y_train)

# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe.ranking_)))

# Print the features that are not eliminated
print(X.columns[rfe.support_])

# Calculates the test set accuracy
acc = accuracy_score(y_test, rfe.predict(X_test_std))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
{'pregnant': 1, 'glucose': 1, 'diastolic': 5, 'triceps': 2, 'insulin': 6, 'bmi': 4, 'family': 1, 'age': 3}
Index(['pregnant', 'glucose', 'family'], dtype='object')
78.8% accuracy on test set.


In [22]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, 0.3)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [24]:
print(accuracy_score(y_test, rf.predict(X_test)))

0.9950576606260296


In [25]:
print(rf.feature_importances_)

[0.00154633 0.00086357 0.00045535 0.00073979 0.00078243 0.00354988
 0.0049853  0.11099988 0.00199698 0.00728055 0.01240843 0.03025133
 0.00078188 0.00279489 0.00147059 0.01370766 0.00393419 0.00061608
 0.00263703 0.00182259 0.00208289 0.00686943 0.00103596 0.00122278
 0.00588273 0.03615508 0.00042108 0.00863875 0.00078667 0.00094452
 0.00073667 0.00213626 0.00057075 0.00708263 0.00326338 0.02455007
 0.00444625 0.03673032 0.00790558 0.00162086 0.00052534 0.03855737
 0.04173666 0.00055813 0.00097183 0.00102382 0.00055843 0.0122277
 0.00062119 0.0143125  0.02520539 0.00052106 0.00052685 0.00616485
 0.0115588  0.00038734 0.00043615 0.00111944 0.01427075 0.00493086
 0.00206494 0.1438532  0.10619575 0.00544109 0.00041621 0.00284787
 0.00613423 0.05525179 0.00084518 0.00154307 0.00302382 0.03503822
 0.00056371 0.00586678 0.00076146 0.00173843 0.01001096 0.00100699
 0.0005667  0.00066962 0.00042994 0.00061887 0.00172817 0.0223065
 0.00130516 0.00187261 0.0006996  0.00062953 0.00085361 0.027999

In [26]:
mask = rf.feature_importances_ > 0.03
print(mask)

[False False False False False False False  True False False False  True
 False False False False False False False False False False False False
 False  True False False False False False False False False False False
 False  True False False False  True  True False False False False False
 False False False False False False False False False False False False
 False  True  True False False False False  True False False False  True
 False False False False False False False False False False False False
 False False False False False False False False False False]


In [27]:
X_reduced = X_train.loc[:, mask]
print(X_reduced.columns)

Index(['biacromialbreadth', 'bimalleolarbreadth', 'chestheight',
       'forearmcircumferenceflexed', 'handbreadth', 'handcircumference',
       'neckcircumference', 'neckcircumferencebase', 'shouldercircumference',
       'sleevelengthspinewrist'],
      dtype='object')


### RFE with random forests

In [28]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=RandomForestClassifier(), 
          n_features_to_select=6, step=10, 
            verbose=1)

rfe.fit(X_train, y_train)

Fitting estimator with 94 features.
Fitting estimator with 84 features.
Fitting estimator with 74 features.
Fitting estimator with 64 features.
Fitting estimator with 54 features.
Fitting estimator with 44 features.
Fitting estimator with 34 features.
Fitting estimator with 24 features.
Fitting estimator with 14 features.


RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=None,
                                     verbose=0, warm_start=False),
    n_features_to_select=6, step=10, verbose=1)

In [29]:
print(X_train.columns[rfe.support_])

Index(['biacromialbreadth', 'handcircumference', 'neckcircumference',
       'neckcircumferencebase', 'shouldercircumference', 'wristcircumference'],
      dtype='object')


In [30]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

In [31]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

# Calculate the accuracy
acc = accuracy_score(y_test, rf.predict(X_test))

# Print the importances per feature
print(dict(zip(X_train.columns, rf.feature_importances_.round(2))))

# Print accuracy
print("{0:.1%} accuracy on test set.".format(acc))

{'pregnant': 0.08, 'glucose': 0.26, 'diastolic': 0.06, 'triceps': 0.08, 'insulin': 0.16, 'bmi': 0.1, 'family': 0.1, 'age': 0.15}
74.6% accuracy on test set.


In [32]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15

# Prints out the mask
print(mask)

[False  True False False  True False False  True]


In [33]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15

# Apply the mask to the feature dataset X
reduced_X = X_train.loc[:, mask]

# prints out the selected column names
print(reduced_X.columns)

Index(['glucose', 'insulin', 'age'], dtype='object')


In [34]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

In [35]:
# Wrap the feature eliminator around the random forest model
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=2, step=2, verbose=1)

# Fit the model to the training data
rfe.fit(X_train, y_train)

# Create a mask using an attribute of rfe
mask = rfe.support_

# Apply the mask to the feature dataset X and print the result
reduced_X = X.loc[:, mask]
print(reduced_X.columns)

Fitting estimator with 8 features.
Fitting estimator with 6 features.
Fitting estimator with 4 features.
Index(['glucose', 'insulin'], dtype='object')


In [36]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, 0.3, 'reg')

In [37]:
from sklearn.linear_model import Lasso

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)

# Create the Lasso model
la = Lasso()

# Fit it to the standardized training data
la.fit(X_train_std, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [39]:
# Transform the test set with the pre-fitted scaler
X_test_std = scaler.transform(X_test)

# Calculate the coefficient of determination (R squared) on X_test_std
r_squared = la.score(X_test_std, y_test)
print("The model can predict {0:.1%} of the variance in the test set.".format(r_squared))

# Create a list that has True values when coefficients equal 0
zero_coef = la.coef_ == 0

# Calculate how many features have a zero coefficient
n_ignored = sum(zero_coef)
print("The model has ignored {} out of {} features.".format(n_ignored, len(la.coef_)))

The model can predict 82.9% of the variance in the test set.
The model has ignored 83 out of 93 features.


In [40]:
alphas = [1, 0.5, 0.1, 0.01]
for a in alphas:
    # Find the highest alpha value with R-squared above 98%
    la = Lasso(alpha=a, random_state=0)

    # Fits the model and calculates performance stats
    la.fit(X_train_std, y_train)
    r_squared = la.score(X_test_std, y_test)
    n_ignored_features = sum(la.coef_ == 0)

    # Print peformance stats 
    print("The model can predict {0:.1%} of the variance in the test set.".format(r_squared))
    print("{} out of {} features were ignored.".format(n_ignored_features, len(la.coef_)))

The model can predict 82.9% of the variance in the test set.
83 out of 93 features were ignored.
The model can predict 90.8% of the variance in the test set.
82 out of 93 features were ignored.
The model can predict 98.5% of the variance in the test set.
69 out of 93 features were ignored.
The model can predict 99.3% of the variance in the test set.
53 out of 93 features were ignored.


### Combining feature selectors 
#### Taking a step back
- Random forest is combination of decision trees 
- We can use combination of models for feature selection too

In [41]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, 0.25, 'reg')

In [42]:
from sklearn.linear_model import LassoCV

lcv = LassoCV()
lcv.fit(X_train, y_train)

lcv.score(X_test, y_test)

0.9895676219048437

In [43]:
lcv_mask = lcv.coef_ != 0
sum(lcv_mask)

38

In [44]:
from sklearn.ensemble import RandomForestRegressor

rfe_rf = RFE(estimator=RandomForestRegressor(),
            n_features_to_select=38, step=5, verbose=5)

rfe_rf.fit(X_train, y_train)

Fitting estimator with 93 features.
Fitting estimator with 88 features.
Fitting estimator with 83 features.
Fitting estimator with 78 features.
Fitting estimator with 73 features.
Fitting estimator with 68 features.
Fitting estimator with 63 features.
Fitting estimator with 58 features.
Fitting estimator with 53 features.
Fitting estimator with 48 features.
Fitting estimator with 43 features.


RFE(estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                    criterion='mse', max_depth=None,
                                    max_features='auto', max_leaf_nodes=None,
                                    max_samples=None, min_impurity_decrease=0.0,
                                    min_impurity_split=None, min_samples_leaf=1,
                                    min_samples_split=2,
                                    min_weight_fraction_leaf=0.0,
                                    n_estimators=100, n_jobs=None,
                                    oob_score=False, random_state=None,
                                    verbose=0, warm_start=False),
    n_features_to_select=38, step=5, verbose=5)

In [45]:
rf_mask = rfe_rf.support_

In [46]:
from sklearn.ensemble import GradientBoostingRegressor

rfe_gb = RFE(estimator=GradientBoostingRegressor(), 
            n_features_to_select=38, step=5, verbose=1)

rfe_gb.fit(X_train, y_train)

Fitting estimator with 93 features.
Fitting estimator with 88 features.
Fitting estimator with 83 features.
Fitting estimator with 78 features.
Fitting estimator with 73 features.
Fitting estimator with 68 features.
Fitting estimator with 63 features.
Fitting estimator with 58 features.
Fitting estimator with 53 features.
Fitting estimator with 48 features.
Fitting estimator with 43 features.


RFE(estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                        criterion='friedman_mse', init=None,
                                        learning_rate=0.1, loss='ls',
                                        max_depth=3, max_features=None,
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_iter_no_change=None,
                                        presort='deprecated', random_state=None,
                                        subsample=1.0, tol=0.0001,
                                        validation_fraction=0.1, verbose=0,
                                        warm_start=False),
    n_f

In [47]:
gb_mask = rfe_gb.support_

In [48]:
votes =np.sum([lcv_mask, rf_mask, gb_mask], axis=0)
print(votes)

[3 1 0 1 3 0 0 0 3 0 3 0 2 0 1 3 3 1 0 1 3 3 0 3 3 1 1 1 1 0 0 0 1 2 1 0 0
 3 3 0 1 0 1 0 0 0 0 1 1 0 0 3 1 2 0 0 0 1 0 3 0 3 0 1 0 2 0 3 0 0 3 1 0 1
 2 2 3 3 1 3 0 2 1 1 0 3 3 1 3 0 2 3 2]


In [49]:
mask = votes >= 2

In [52]:
reduced_X = X.loc[:, mask]

In [60]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, 0.3, 'reg')

Dataset is different than the one in the datacamp exercise.

In [61]:
from sklearn.linear_model import LassoCV

# Create and fit the LassoCV model on the training set
lcv = LassoCV()
lcv.fit(X_train, y_train)
print('Optimal alpha = {0:.3f}'.format(lcv.alpha_))

# Calculate R squared on the test set
r_squared = lcv.score(X_test, y_test)
print('The model explains {0:.1%} of the test set variance'.format(r_squared))

# Create a mask for coefficients not equal to zero
lcv_mask = lcv.coef_ != 0
print('{} features out of {} selected'.format(sum(lcv_mask), len(lcv_mask)))

Optimal alpha = 0.406
The model explains 98.9% of the test set variance
38 features out of 93 selected


In [62]:
# Select 10 features with RFE on a GradientBoostingRegressor, drop 20 features on each step
rfe_gb = RFE(estimator=GradientBoostingRegressor(), 
             n_features_to_select=10, step=20, verbose=1)
rfe_gb.fit(X_train, y_train)

Fitting estimator with 93 features.
Fitting estimator with 73 features.
Fitting estimator with 53 features.
Fitting estimator with 33 features.
Fitting estimator with 13 features.


RFE(estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                        criterion='friedman_mse', init=None,
                                        learning_rate=0.1, loss='ls',
                                        max_depth=3, max_features=None,
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_iter_no_change=None,
                                        presort='deprecated', random_state=None,
                                        subsample=1.0, tol=0.0001,
                                        validation_fraction=0.1, verbose=0,
                                        warm_start=False),
    n_f

In [63]:
# Calculate the R squared on the test set
r_squared = rfe_gb.score(X_test, y_test)
print('The model can explain {0:.1%} of the variance in the test set'.format(r_squared))

The model can explain 97.1% of the variance in the test set


In [64]:
# Assign the support array to gb_mask
gb_mask = rfe_gb.support_

In [65]:
# Select 10 features with RFE on a RandomForestRegressor, drop 20 features on each step
rfe_rf = RFE(estimator=RandomForestRegressor(), 
             n_features_to_select=10, step=20, verbose=1)
rfe_rf.fit(X_train, y_train)

Fitting estimator with 93 features.
Fitting estimator with 73 features.
Fitting estimator with 53 features.
Fitting estimator with 33 features.
Fitting estimator with 13 features.


RFE(estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                    criterion='mse', max_depth=None,
                                    max_features='auto', max_leaf_nodes=None,
                                    max_samples=None, min_impurity_decrease=0.0,
                                    min_impurity_split=None, min_samples_leaf=1,
                                    min_samples_split=2,
                                    min_weight_fraction_leaf=0.0,
                                    n_estimators=100, n_jobs=None,
                                    oob_score=False, random_state=None,
                                    verbose=0, warm_start=False),
    n_features_to_select=10, step=20, verbose=1)

In [67]:
# Calculate the R squared on the test set
r_squared = rfe_rf.score(X_test, y_test)
print('The model can explain {0:.1%} of the variance in the test set'.format(r_squared))

The model can explain 97.5% of the variance in the test set


In [68]:
# Assign the support array to gb_mask
rf_mask = rfe_rf.support_

In [70]:
# Sum the votes of the three models
votes = np.sum([lcv_mask, gb_mask, rf_mask], axis=0)
print(votes)

[2 0 0 0 3 0 0 0 3 0 1 0 1 0 0 1 3 0 0 0 3 1 0 3 2 0 0 1 1 0 0 0 1 0 0 0 0
 1 1 0 1 0 0 0 0 0 0 1 0 0 0 2 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 1 0 1
 1 0 3 1 1 1 0 0 1 1 0 3 1 0 1 0 0 2 2]


In [72]:
# Create a mask for features selected by all 3 models
meta_mask = votes >= 3
print(meta_mask)

[False False False False  True False False False  True False False False
 False False False False  True False False False  True False False  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False  True False False False False False False False]


In [75]:
# Apply the dimensionality reduction on X
X_reduced = X.iloc[:, meta_mask]
print(X_reduced.columns)

Index(['axillaheight', 'bicepscircumferenceflexed', 'buttockdepth',
       'calfcircumference', 'chestcircumference', 'thighcircumference',
       'waistcircumference'],
      dtype='object')


In [77]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

# Plug the reduced dataset into a linear regression pipeline
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=0)
lm.fit(scaler.fit_transform(X_train), y_train)
r_squared = lm.score(scaler.transform(X_test), y_test)
print('The model can explain {0:.1%} of the variance in the test set using {1:} features.'.format(r_squared, len(lm.coef_)))

The model can explain 97.1% of the variance in the test set using 7 features.


In [ ]:
!jupyter nbconvert --to html 3_Screening_for_model_accuracy.ipynb

In [ ]:
!../gitbsh > /dev/null 2>&1